In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn import metrics
#import data from files

x_header = pd.read_csv("x_header.csv")
train_data = pd.read_csv('train_data.csv', header = None)
test_data = pd.read_csv('test_data.csv', header = None)
train_labels = pd.read_csv('train_labels.csv', header = None, dtype = 'category' )


#create variables 
header = pd.DataFrame(data = x_header)
X = pd.DataFrame(data = train_data)
X.columns = header.ID
X_test = pd.DataFrame(data = test_data)
X_test.columns = header.ID
y = pd.DataFrame(data = train_labels)
y.columns = ['Genre']

print("Part 1 done...")

Part 1 done...


In [2]:
#Merge X and X_test

X = pd.concat([X,X_test])

#View value frequencies from each columns 
#for i in X.columns:
#    print("Value count", i, ": ", X[i].value_counts().head(5))
#Drop bad data

X = X.drop(columns = ['MFCC_Mean_1','MFCC_Mean_2','MFCC_Mean_3','MFCC_Mean_4','MFCC_Min_1'])
#Normalize data according to group
#display(HTML(x_y_train.head(15).to_html(max_rows=15)))

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X)
X = scaler.transform(X)

X_test = X[4363:]
X = X[:4363]

print("Part 2 done...")

Part 2 done...


In [3]:
#Split data to test-validation set
def run_model(model, alg_name, test_size):
    global X
    global y
    global X_train, X_val, y_train, y_val
    y = np.ravel(y)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)
    model.fit(X_train, y_train)
    print("----",alg_name,"----")
    print(model.score(X_train,y_train))
    print(model.score(X_val,y_val))
    #Apply cross validation
    scores = cross_validate(model, X_train, y_train, cv=3)
    print("Cross-validated scores:", scores["test_score"])

#try this  https://www.kaggle.com/coolman/different-classification-techniques-python
#Neural network results highest score
# ----------- Neural network - Multi-layer Perceptron  ------------
#from sklearn.neural_network import MLPClassifier

#model = MLPClassifier()
#run_model(model, " MLP Neural network default", 0.2)

#model1 = MLPClassifier()
#run_model(model1, " MLP Neural network default", 0.3)


In [ ]:
y_test_pred = model.predict(X_test)
indx = range(y_test_pred.shape[0])
result = pd.DataFrame(index = indx, columns = ["Sample_id","Sample_label"])
result["Sample_id"] = result.index+1
result["Sample_label"] = y_test_pred
result.to_csv("MLPClassifier_MinMaxScaler_test20.csv", index = False)

y_test_pred = model1.predict(X_test)
indx = range(y_test_pred.shape[0])
result = pd.DataFrame(index = indx, columns = ["Sample_id","Sample_label"])
result["Sample_id"] = result.index+1
result["Sample_label"] = y_test_pred
result.to_csv("MLPClassifier__MinMaxScaler_test30.csv", index = False)



In [5]:
#Other models 

# ----- Random Forest ---------------
from sklearn.ensemble import RandomForestClassifier

model4 = RandomForestClassifier(n_estimators=15)
run_model(model4, "Random Forest", 0.25)

# ----- xgboost ------------
from xgboost import XGBClassifier

modelxg = XGBClassifier(max_depth = 3, objective = 'multi:softmax')
run_model(modelxg, "XGBoost",0.25)

---- Random Forest ----
0.994193154034
0.597616865261
Cross-validated scores: [ 0.58096981  0.5472044   0.59007353]
---- XGBoost ----
0.880195599022
0.658111824015
Cross-validated scores: [ 0.63129003  0.61136572  0.61672794]


In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_val)
conf_mat = confusion_matrix(y_true=y_val, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

labels = ['Class 0', 'Class 1']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

y_pred = model1.predict(X_val)
conf_mat = confusion_matrix(y_true=y_val, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

labels = ['Class 0', 'Class 1']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()